In [ ]:
import pandas as pd
from google.colab import files

# Upload do arquivo
uploaded = files.upload()

# Carregamento do dataset
df = pd.read_csv(list(uploaded.keys())[0])
df.columns = df.columns.str.strip()  # Remover espaços dos nomes das colunas

# Ver os primeiros dados
df.head()

Saving vendas_modificado.csv to vendas_modificado (4).csv


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,3/20/2021,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,10/30/2020,9:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,6/9/2021,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,6/4/2022,8:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,5/4/2019,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


**Limpeza basica:**

In [ ]:
# Verificar nulos
print("Valores nulos por coluna:")
print(df.isnull().sum())

# Remover duplicatas
df.drop_duplicates(inplace=True)

# Cria uma cópia antes da conversão numérica
df['valor_limpo'] = df['valor'].str.replace('R\$', '', regex=True).str.strip()

# Filtra os que dariam NaN
problemas = df.loc[pd.to_numeric(df['valor_limpo'], errors='coerce').isna(), 'valor_limpo']
print(problemas.unique())

# Tratar nulos com valores padrão (ajuste conforme necessidade)
df['cliente'].fillna('Desconhecido', inplace=True)
df['valor'].fillna(0, inplace=True)
df['quantidade'].fillna(1, inplace=True)
df['frete'].fillna(0, inplace=True)
df['status'].fillna('indefinido', inplace=True)

Valores nulos por coluna:
id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete           7371
pagamento          0
vendedor        3680
marca              0
dtype: int64
['16,87' '3,25' '1,63' ... '37,15' '153,26999999999998'
 '2022,6399999999999']


<ipython-input-125-d178841e3a39>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['cliente'].fillna('Desconhecido', inplace=True)
<ipython-input-125-d178841e3a39>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

**Padronização e Correção**

In [ ]:
# 1. Remover "R$", espaços extras e substituir vírgulas por pontos (caso haja valores como "R$ 15,90")
df['valor'] = df['valor'].astype(str)  # garantir que tudo é string
df['valor'] = df['valor'].str.replace('R\$', '', regex=True)
df['valor'] = df['valor'].str.replace(',', '.', regex=False)
df['valor'] = df['valor'].str.strip()

# 2. Converter para numérico
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')

# 3. Verificar se ainda há valores NaN
print("Valores NaN:", df['valor'].isnull().sum())

Valores NaN: 0


In [ ]:
# Remove linhas com valor NaN na coluna 'valor'
df = df.dropna(subset=['valor'])

# Confere se deu certo
print(df['valor'].isna().sum())  # Deve retornar 0


0


In [ ]:
print(df['valor'])

0          16.87
1           3.25
2           1.63
3           8.74
4           9.48
           ...  
368747      1.43
368748    682.51
368749     11.01
368750      4.56
368751      3.17
Name: valor, Length: 363511, dtype: float64


In [ ]:
# Datas e hora
df['data'] = pd.to_datetime(df['data'], errors='coerce')
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.time

In [ ]:
# Padronizado textos
colunas_texto = ['cidade', 'estado', 'pais', 'status', 'pagamento', 'vendedor', 'marca']

for col in colunas_texto:
    df[col] = df[col].astype(str).str.strip().str.lower()

In [ ]:
# Ver os uniques na coluna cidade

df['cidade'].unique()

array(['niterói', 'mariana', 'cabo frio', 'campos dos goytacazes',
       'conselheiro lafaiete', 'resende costa', 'são paulo', 'bicas',
       'são joão del-rei', 'são gonçalo', 'são bernardo do campo',
       'duque de caxias', 'macaé', 'viçosa', 'juiz de fora', 'guarulhos',
       'matias barbosa', 'campinas', 'barroso', 'carandaí', 'barbacena',
       'itaboraí', 'muriaé', 'petrópolis', 'ouro branco', 'prados',
       'santos dumont', 'cataguases', 'são joão de meriti', 'ubá',
       'palma', 'simão pereira', 'magé', 'ewbank da câmara',
       'santana do garambéu', 'astolfo dutra', 'são joão nepomuceno',
       'belo horizonte', 'tocantins', 'congonhas', 'são vicente de minas',
       'coronel xavier chaves', 'leopoldina', 'belford roxo', 'ritápolis',
       'lagoa dourada', 'rio de janeiro', 'nova iguaçu', 'são tiago',
       'são josé dos campos', 'volta redonda', 'lima duarte',
       'santo andré', 'tiradentes', 'ouro preto', 'angra dos reis'],
      dtype=object)

In [ ]:
# Ver os uniques na coluna estado

df['estado'].unique()

array(['rj', 'mg', 'sp', 'mtsa', 'são paulo', 'rs', 'pr', 'psc'],
      dtype=object)

In [ ]:
# Correção das siglas

df['estado'] = df['estado'].str.strip().str.lower()

df['estado'] = df['estado'].replace({
    'são paulo': 'sp',
    'mtsa': 'mt',
    'psc': 'sc'
})

In [ ]:
# Ver os uniques na coluna pais

df['pais'].unique()

array(['brasil'], dtype=object)

In [ ]:
# Ver os uniques na coluna status

df['status'].unique()

array(['pagamento confirmado', 'em separação', 'entregue com sucesso',
       'aguardando pagamento', 'em transporte', 'ap', 'entregue', 'pc',
       'sep', 'separando', 'pgto confirmado', 'transp', 'entg',
       'aguardando pgto', 'transportando'], dtype=object)

In [ ]:
# Padronização na coluna status:

df['status'] = df['status'].str.strip().str.lower()

df['status'] = df['status'].replace({
    'pgto confirmado': 'pagamento confirmado',
    'aguardando pgto': 'aguardando pagamento',
    'sep': 'em separação',
    'separando': 'em separação',
    'transp': 'em transporte',
    'transportando': 'em transporte',
    'entregue com sucesso': 'entregue',
    'entg': 'entregue',
    'ap': 'aguardando pagamento',
    'pc': 'pagamento confirmado'
})

In [ ]:
# Ver os uniques na coluna pagamento

df['pagamento'].unique()

array(['cartão de crédito', 'pix', 'transferência bancária', 'boleto',
       'pix à vista', 'cartão crédito à vista', 'pagamento instantâneo',
       'cartão crédito', 'boleto à vista', 'doc', 'ted',
       'boleto no dinheiro'], dtype=object)

In [ ]:
df['pagamento'] = df['pagamento'].str.strip().str.lower()

df['pagamento'] = df['pagamento'].replace({
    'cartão crédito': 'cartão de crédito',
    'cartão crédito à vista': 'cartão de crédito',
    'pix à vista': 'pix',
    'pagamento instantâneo': 'pix',
    'boleto à vista': 'boleto',
    'boleto no dinheiro': 'dinheiro',
    'doc': 'transferência bancária',
    'ted': 'transferência bancária'
})

In [ ]:
# Ver os uniques na coluna vendedor

df['vendedor'].unique()

array(['samuel henrique caçador', 'micael malaquias de souza oliveira',
       'henrico matos lima da cunha', 'gabriel queiroz de aguiar',
       'victor gonçalves donadoni',
       'felipe henrique costa barnabe marazo', 'henrico victor alves',
       'carlos queiroz de aguiar', 'lucas vitor façanha neves',
       'paulo souza roncete', 'nan'], dtype=object)

In [ ]:
# Ver os uniques na coluna marca

df['marca'].unique()

array(['porto alegre', 'fugini', 'minalba', 'marca-brás', '3 corações',
       'caldo bom', 'pantene', 'dove', 'caravelas', 'crystal', 'itambé',
       'neve', 'liza', 'predilecta', 'pilão', 'seda', 'melitta', 'salton',
       'adria', 'coamo', 'aurora', 'skol', 'união', 'scott', 'miolo',
       'oral-b', 'aviação', 'itamarati', 'nutrella', 'sorriso',
       'brilhante', 'elefante', 'snob', 'confort', 'pinho sol',
       'coca-cola', 'nivea', 'lysoform', 'tirolez', 'kicaldo', 'colgate',
       'concha y toro', 'urbano', 'sol', 'fofo', 'triunfo', 'quatá',
       'lebre', 'prats', 'dona benta', 'bauducco', 'lux', 'ypê',
       'dafruta', 'barilla', 'elite', 'tixan', 'elegê', 'tio joão',
       'minuano', 'soya', 'renata', 'dualette', 'wickbold', 'nestlé',
       'qualy', 'cisne', 'pullman', 'marinho', 'fanta', 'limpol',
       'perdigão', 'guaraná antártica', 'veja', 'brahma', 'omo',
       'heineken', 'genérico', 'del valle', 'anaconda', 'personal',
       'sadia', 'namorado', 'são lour

In [ ]:
# Ver os uniques na coluna produto

df['produto'].unique()

array(['Queijo Mussarela', 'Molho de Tomate', 'Água Mineral', 'Carvão',
       'Café', 'Feijão', 'Condicionador', 'Sabonete', 'Açúcar',
       'Manteiga', 'Papel Higiênico', 'Óleo de Soja', 'Vinho', 'Macarrão',
       'Shampoo', 'Presunto', 'Cerveja', 'Papel Toalha', 'Pasta de Dente',
       'Pão de Forma', 'Sabão em Pó', 'Amaciante', 'Desinfetante',
       'Refrigerante', 'Arroz', 'Farinha de Trigo', 'Biscoito Recheado',
       'Sal', 'Suco de Laranja', 'Leite Integral', 'Detergente',
       'Refrigkrante', 'Scl', 'Papel qoalha', 'Desinfetanue',
       'Qbeijo Mussarela', 'Água Mineras', 'Presuntd', 'Macawrão',
       'Sucoyde Laranja', 'Clfé', 'Desinfekante', 'Majarrão', 'Cnfé',
       'Farinha de Tripo', 'Manteigt', 'zabonete', 'Mqcarrão', 'Arroc',
       'tal', 'Sabonepe', 'Papel Twalha', 'Água Mineual', 'Condicioiador',
       'Condibionador', 'Caft', 'ieijão', 'Açúcaz', 'Amaciayte', 'Caff',
       'Mopho de Tomate', 'Molmo de Tomate', 'Biscoitq Recheado',
       'Macirrão', 'Deqe

In [ ]:
# Correcao em produtos
df['produto'] = df['produto'].replace({  # Changed 'produtos' to 'produto'
    'café': ['café', 'cafc', 'caft', 'cnfé', 'clfé', 'caff', 'Café#$@!', 'Clfé', 'Cnfé', 'Caft', 'Caff', 'Cafc'],
    'pão de forma': ['pão de forma', 'Pão de Forma#$@!'],
    'sabão em pó': ['sabão em pó', 'sabonepe', 'Sabão em Pó#$@!'],
    'molho de tomate': ['molho de tomate', 'molmo de tomate', 'mopho de tomate', 'Molho de Tomate#$@!', 'Mopho de Tomate'],
    'macarrão': ['macarrão', 'majarrão', 'macawrão', 'macirrão', 'mqcarrão', 'Macarrão#$@!', 'Macawrão', 'Majarrão', 'Mqcarrão'],
    'desinfetante': ['desinfetante', 'desinfetanue', 'desinfekante', 'Desinfetante#$@!', 'Desinfetanue', 'Desinfekante'],
    'presunto': ['presunto', 'presuntd', 'Presunto#$@!', 'Presuntd'],
    'água mineral': ['água mineral', 'água mineras', 'água mineual','Água Mineral#$@!', 'Água Mineras', 'Água Mineual'],
    'feijão': ['feijão', 'ieijão', 'Feijão#$@!', 'ieijão'],
    'manteiga': ['manteiga', 'manteigt', 'Manteiga#$@!', 'Manteigt'],
    'condicionador': ['condicionador', 'condicioiador', 'condibionador', 'Condicionador#$@!', 'Condicioiador#$@!', 'Condibionador'],
    'biscoito recheado': ['biscoito recheado', 'biscoitq recheado', 'Biscoito Recheado#$@!'],
    'detergente': ['detergente', 'deqergente', 'deterwente', 'Detergente#$@!', 'Deqergente', 'Deterwente'],
    'arroz': ['arroz', 'arroc', 'Arroz#$@!', 'Arroc'],
    'sal': ['sal', 'scl', 'tal', 'Sal#$@!', 'Scl'],
    'papel toalha': ['papel toalha', 'papel twalha', 'papel qoalha', 'Papel Toalha#$@!', 'Papel qoalha', 'Papel Twalha'],
    'queijo mussarela': ['queijo mussarela', 'qbeijo mussarela', 'queijo mussarelz', 'Queijo Mussarela#$@!', 'Qbeijo Mussarela', 'Queijo Mussarelz'],
    'suco de laranja': ['suco de laranja', 'sucoyde laranja', 'Suco de Laranja#$@!', 'Sucoyde Laranja'],
    'açúcar': ['açúcar', 'açúcaz', 'Açúcar#$@!', 'Açúcaz'],
    'amaciante': ['amaciante', 'amaciayte', 'Amaciante#$@!', 'Amaciayte'],
    'refrigerante': ['refrigerante', 'refrigkrante', 'Refrigerante#$@!', 'Refrigkrante'],
    'shampoo': ['shampoo', 'Shampoo#$@!'],
    'sabonete': ['Sabonete#$@!', 'zabonete', 'Sabonepe'],
    'papel higiênico': ['papel higiênico', 'Papel Higiênico#$@!'],
    'pasta de dente': ['pasta de dente', 'Pasta de Dente#$@!'],
    'leite integral': ['leite integral', 'Leite Integral#$@!'],
    'óleo de soja': ['óleo de soja', 'Óleo de Soja#$@!'],
    'farinha de trigo': ['Farinha de Trigo#$@!', 'Farinha de Tripo'],
    'cerveja': ['cerveja', 'Cerveja#$@!'],
    'vinho': ['vinho', 'Vinho#$@!'],
    'carvão': ['Carvão', 'Carvão#$@!']
})

In [ ]:
# Corrigindo valores com caracteres especiais
for produto in df['produto'].unique():
    if "#$@!" in produto:
        df['produto'] = df['produto'].replace({produto: produto.replace("#$@!", "")})

# Verificando os valores únicos após a correção
print(df['produto'].unique())

['Queijo Mussarela' 'Molho de Tomate' 'Água Mineral' 'Carvão' 'Café'
 'Feijão' 'Condicionador' 'Sabonete' 'Açúcar' 'Manteiga' 'Papel Higiênico'
 'Óleo de Soja' 'Vinho' 'Macarrão' 'Shampoo' 'Presunto' 'Cerveja'
 'Papel Toalha' 'Pasta de Dente' 'Pão de Forma' 'Sabão em Pó' 'Amaciante'
 'Desinfetante' 'Refrigerante' 'Arroz' 'Farinha de Trigo'
 'Biscoito Recheado' 'Sal' 'Suco de Laranja' 'Leite Integral' 'Detergente'
 'Refrigkrante' 'Scl' 'Papel qoalha' 'Desinfetanue' 'Qbeijo Mussarela'
 'Água Mineras' 'Presuntd' 'Macawrão' 'Sucoyde Laranja' 'Clfé'
 'Desinfekante' 'Majarrão' 'Cnfé' 'Farinha de Tripo' 'Manteigt' 'zabonete'
 'Mqcarrão' 'Arroc' 'tal' 'Sabonepe' 'Papel Twalha' 'Água Mineual'
 'Condicioiador' 'Condibionador' 'Caft' 'ieijão' 'Açúcaz' 'Amaciayte'
 'Caff' 'Mopho de Tomate' 'Molmo de Tomate' 'Biscoitq Recheado' 'Macirrão'
 'Deqergente' 'Cafc' 'Queijo Mussarelz' 'Deterwente']


In [ ]:
# Ver os uniques na coluna valor

df['frete'].unique()

array([ 20. ,   0. ,  22. ,  10. ,   9. ,  10.5,  18. ,  19. ,  13. ,
         8.5,  18.5,  12. ,   8. ,  11. ,  21. ,   9.5, 100. ,  17. ,
        95. ,  23. ,  12.5,  80. ,  90. ,  85. ])

In [ ]:
import numpy as np

# Multiplicar valores negativos por -1
df['frete'] = df['frete'].abs()  # Usando abs() para tornar todos os valores positivos

# Definir limite superior para o frete (ajuste conforme necessário)
limite_superior = 100

# Substituir valores acima do limite pelo limite
df['frete'] = np.where(df['frete'] > limite_superior, limite_superior, df['frete'])

# Verificar os valores únicos após a correção
print(df['frete'].unique())

[ 20.    0.   22.   10.    9.   10.5  18.   19.   13.    8.5  18.5  12.
   8.   11.   21.    9.5 100.   17.   95.   23.   12.5  80.   90.   85. ]


In [ ]:
df['cliente'] = df['cliente'].str.upper()
df['vendedor'] = df['vendedor'].str.upper()

In [ ]:
df.head()

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,frete,pagamento,vendedor,marca,valor_limpo
0,13679,2021-03-20,23:35:51,LUCAS ARAUJO KUHN,Queijo Mussarela,16.87,13,239.31,pagamento confirmado,niterói,rj,brasil,20.0,cartão de crédito,SAMUEL HENRIQUE CAÇADOR,porto alegre,"16,87"
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho De Tomate,3.25,3,9.75,pagamento confirmado,mariana,mg,brasil,0.0,pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,fugini,"3,25"
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,pagamento confirmado,cabo frio,rj,brasil,22.0,transferência bancária,HENRICO MATOS LIMA DA CUNHA,minalba,"1,63"
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,em separação,campos dos goytacazes,rj,brasil,20.0,pix,GABRIEL QUEIROZ DE AGUIAR,marca-brás,"8,74"
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,em separação,conselheiro lafaiete,mg,brasil,0.0,cartão de crédito,HENRICO MATOS LIMA DA CUNHA,3 corações,"9,48"


In [ ]:
df.dropna(inplace=True)

df.drop_duplicates(inplace=True)

df['produto'] = df['produto'].str.title().str.strip()

df.drop(columns=['cep'], inplace=True)


In [ ]:
# Nome do arquivo CSV de saída
nome_arquivo = 'novo_dataset_limpo.csv'

# Exportar o DataFrame para CSV
df.to_csv(nome_arquivo, index=False)

# Fazer o download do arquivo (no ambiente do Colab)
from google.colab import files
files.download(nome_arquivo)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>